# Imports

In [225]:
#!pip install -U scikit-learn

In [226]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean

# Input

In [227]:
transformation = Transformation()

user_number_points = 120
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [228]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [229]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [230]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [231]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.03995546 0.0399787  0.03995546 ... 0.03719718 0.05399474 0.03719718]
[0.36806863 0.36828277 0.36806863 ... 0.3426595  0.49739818 0.3426595 ]
[0.66490239 0.66499121 0.66490239 ... 0.65428205 0.71626022 0.65428205]
(5999,)


### Multinomial Sampling

In [232]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 1 1 0]


In [233]:
target_number_point = min(len(graph._node), user_number_points)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[ 75.12734  ,   1.4916434,  68.3      ],
       [ 78.490616 ,   8.610001 ,  87.24593  ],
       [ 51.       ,  -6.3966537,  96.04341  ],
       [ 82.1445   ,   8.642877 ,  68.3      ],
       [ 82.76066  ,   2.9401166, -11.4311495]], dtype=float32)

# KNN

In [257]:
_, indices = NearestNeighbors(n_neighbors=10).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 120
Number of edges: 694


# Edge Predictor

In [235]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(120, 64)

In [236]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [237]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

# Face Candidates

#### Inputs

In [238]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 7)	1
  (1, 9)	1
  (1, 49)	1
  (1, 67)	1
  (1, 68)	1
  (1, 69)	1
  (1, 80)	1
  (2, 0)	1
  (2, 1)	1
  (2, 4)	1
  (2, 5)	1
  (2, 6)	1
  (2, 8)	1
  :	:
  (117, 95)	1
  (117, 97)	1
  (117, 99)	1
  (117, 100)	1
  (117, 101)	1
  (117, 102)	1
  (117, 119)	1
  (118, 42)	1
  (118, 70)	1
  (118, 71)	1
  (118, 77)	1
  (118, 78)	1
  (118, 112)	1
  (118, 113)	1
  (118, 114)	1
  (118, 115)	1
  (119, 60)	1
  (119, 100)	1
  (119, 101)	1
  (119, 102)	1
  (119, 112)	1
  (119, 113)	1
  (119, 114)	1
  (119, 115)	1
  (119, 117)	1
[[9.20356427e-04 7.03309305e-04 6.00670143e-04 ... 2.17676777e-02
  7.07031418e-03 4.32298320e-03]
 [1.36095630e+06 4.90790187e-02 4.76539001e-02 ... 8.22336070e-02
  7.11326590e-02 6.63244706e-02]
 [2.85194631e+06 1.26632573e-02 1.07180269e-03 ... 2.82255844e-02
  1.02070303e-02 6.52148795e-03]
 ...
 [4.35328471e+02 1.25528131e+02 4.51458450e+02

In [239]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0

A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T

A_s = A_s/A_s.max()                                         # Normalize

In [240]:
print(A_s)  # symmétrique
print(A_s.shape)

[[1.99409875e-57 1.59392857e-52 3.33989914e-52 ... 1.43335724e-51
  1.45488785e-49 6.26518308e-49]
 [1.59392857e-52 3.05422897e-50 3.27554740e-50 ... 1.14344916e-45
  3.34900723e-43 2.18324819e-43]
 [3.33989914e-52 3.27554740e-50 3.16088507e-51 ... 2.39611519e-45
  7.01794854e-43 4.57494684e-43]
 ...
 [1.43335724e-51 1.14344916e-45 2.39611519e-45 ... 1.70462081e-44
  2.11890610e-42 8.21394796e-42]
 [1.45488785e-49 3.34900723e-43 7.01794854e-43 ... 2.11890610e-42
  3.20569539e-40 1.00643599e-39]
 [6.26518308e-49 2.18324819e-43 4.57494684e-43 ... 8.21394796e-42
  1.00643599e-39 3.94867618e-39]]
(120, 120)


# Face Classifier

### TriConv

#### Inputs

In [241]:
triangles = list(nx.simple_cycles(extended_graph, length_bound=3))  # [triangle0, triangle1,...] | triangle0 = [node1,node2,node3] | node1 = (x ,y ,z)
# triangles = cycles
print(triangles[0])
print(np.array(triangles).shape)    #nb_triangle, 3 nodes, 3 dimensions par node

[(75.12734, 1.4916434, 68.3), (76.860954, -5.4119864, 68.3), (82.1445, 8.642877, 68.3)]
(1656, 3, 3)


In [242]:
p_init = np.zeros((len(triangles)))

for index_triangle, triangle in enumerate(triangles):
    i = list(dict(extended_graph._node).keys()).index(triangle[0])
    j = list(dict(extended_graph._node).keys()).index(triangle[1])
    k = list(dict(extended_graph._node).keys()).index(triangle[2])
    p_init[index_triangle] = (A_s[i,j] + A_s[i,k] + A_s[j,k])/3
print(p_init)
print(len(p_init))

[1.10829522e-50 1.27894462e-49 4.02076493e-50 ... 2.90087936e-40
 2.29156066e-40 1.75044014e-41]
1656


#### Calculate barycenter

In [243]:
barycenters = list()

for _, triangle in enumerate(triangles):
    b_x = (triangle[0][0] + triangle[1][0] + triangle[2][0]) / 3
    b_y = (triangle[0][1] + triangle[1][1] + triangle[2][1]) / 3
    b_z = (triangle[0][2] + triangle[1][2] + triangle[2][2]) / 3
    barycenters.append([b_x, b_y, b_z])

print(barycenters[0])
print(len(barycenters))

[78.04426574707031, 1.574177900950114, 68.30000305175781]
1656


#### KNN Tri

In [244]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)


Diff Barycenters

In [245]:
barycenters_array = np.array(barycenters)
indices_neigh_tri_array = np.array(indices_neigh_tri)

# Calculate differences between points
barycenters_diff = np.subtract(barycenters_array[indices_neigh_tri_array[:, 0]][:, np.newaxis], barycenters_array[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(1656, 19, 3)


#### calculate e norm matrix

In [246]:
diff_vectors = list()

for _, triangle in enumerate(triangles):
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])
print(diff_vectors)
print(len(diff_vectors))

[[7.117971, 10.019014, 15.015161], [7.117971, 14.446762, 12.917183], [7.117971, 13.698585, 13.170287], [7.117971, 12.901069, 14.660348], [10.019014, 13.933014, 14.530291], [10.019014, 13.170287, 15.034595], [10.019014, 13.229373, 15.193528], [10.019014, 13.053234, 15.34758], [12.901069, 13.229373, 2.6546915], [12.901069, 13.933014, 4.6093416], [13.053234, 14.446762, 8.314937], [13.053234, 13.698585, 6.5037603], [13.053234, 13.170287, 0.66601205], [13.170287, 14.446762, 8.895977], [13.170287, 13.698585, 7.1179705], [13.229373, 13.933014, 1.9954203], [13.698585, 14.446762, 1.9942145], [12.917183, 13.170287, 1.9942145], [9.476505, 1.8094056, 8.317948], [9.476505, 1.4871291, 8.899291], [9.476505, 1.3214964, 8.900156], [9.476505, 2.0334368, 9.465697], [9.476505, 0.61642694, 9.481483], [6.5760055, 4.437221, 2.9112444], [6.5760055, 3.5314927, 3.0869265], [6.5760055, 1.8702312, 4.897841], [6.5760055, 2.7053819, 5.0373836], [6.5760055, 1.4317573, 5.56708], [6.5760055, 0.61642694, 6.081328], [6.

#### Calculate r

In [247]:
max_diff_vectors = np.array(diff_vectors).max(axis=1)       # calculate t_n_max
min_diff_vectors = np.array(diff_vectors).min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(1656,)

In [248]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(1656, 19)
(1656, 19)


In [249]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))


# Fill r_matrix with the corresponding indices from indices_neigh_tri
r_matrix[:, :, 0] = min_diff_vectors_diff
r_matrix[:, :, 1] = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(1656, 19, 5)


#### Calculate f

In [250]:
# MLP * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999925
(1656,)
[0.00060049 0.00060049 0.00060049 ... 0.00060049 0.00060049 0.00060049]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_3912\1064855114.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [251]:
selected_triangles_indexes = np.argpartition(final_scores, -target_number_point)[-target_number_point:] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)

(120, 3, 3)


In [252]:
def create_triangle_graph(XYZ):
    extended_graph = nx.Graph()
    for index_poly, poly in enumerate(XYZ):
        for index_current_node in range(len(poly)):
            current_node = tuple(poly[index_current_node])
            for index_other_node in range(index_current_node+1, len(poly)):
                edge = current_node, tuple(poly[index_other_node])
                extended_graph.add_edge(*edge)
                # if attribute do not exists
                if len(extended_graph.nodes[current_node])==0:
                    extended_graph.nodes[current_node]['index_triangle'] = set()
                extended_graph.nodes[current_node]['index_triangle'].add(index_poly)
                if len(extended_graph.nodes[tuple(poly[index_other_node])])==0:
                    extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
                extended_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
    return extended_graph

In [253]:
selected_triangles

array([[[76.597786  , -7.328478  , 84.78869   ],
        [76.611404  , -7.6759295 , 85.39784   ],
        [77.309     , -7.328478  , 84.37807   ]],

       [[76.597786  , -7.328478  , 84.78869   ],
        [76.611404  , -7.6759295 , 85.39784   ],
        [77.629845  , -6.5098743 , 82.37088   ]],

       [[76.597786  , -7.328478  , 84.78869   ],
        [76.611404  , -7.6759295 , 85.39784   ],
        [78.34155   , -7.6759295 , 84.20875   ]],

       ...,

       [[76.82386   ,  7.9794145 , -0.95131457],
        [39.        ,  8.793697  , -4.666995  ],
        [76.57269   ,  7.9794145 , -1.0402565 ]],

       [[76.82386   ,  7.9794145 , -0.95131457],
        [39.        ,  8.793697  , -4.666995  ],
        [75.38672   ,  2.6241417 ,  3.696378  ]],

       [[39.        ,  5.145526  , 97.13888   ],
        [51.        , -5.921278  , 96.50987   ],
        [39.        , -7.9077644 , 93.86099   ]]], dtype=float32)

In [254]:
simplified_final_graph = create_triangle_graph(selected_triangles)
print(simplified_final_graph._node)

{(76.597786, -7.328478, 84.78869): {'index_triangle': {0, 1, 2, 3, 4, 5, 6, 8, 10, 12, 13, 14, 15, 16, 20, 21, 33, 61, 77}}, (76.611404, -7.6759295, 85.39784): {'index_triangle': {0, 1, 2, 3, 4, 5, 10, 13, 14, 18, 21, 25, 37, 38, 39, 40, 44, 55, 61, 62, 68, 72, 73, 77, 80, 88, 116}}, (77.309, -7.328478, 84.37807): {'index_triangle': {0, 38, 15}}, (77.629845, -6.5098743, 82.37088): {'index_triangle': {1}}, (78.34155, -7.6759295, 84.20875): {'index_triangle': {2, 6, 39}}, (78.98692, -8.447791, 86.12344): {'index_triangle': {40, 3, 12}}, (78.09511, -6.9390492, 82.578026): {'index_triangle': {4}}, (78.1065, -8.722893, 88.17731): {'index_triangle': {8, 44, 5}}, (76.95771, -8.447791, 87.22521): {'index_triangle': {6, 8, 12, 15, 16, 20, 25, 33, 37, 38, 39, 40, 44, 55, 62, 68, 76, 77, 86, 89, 116}}, (51.0, -6.3966537, 96.04341): {'index_triangle': {36, 7, 72, 55, 89, 94}}, (39.0, -7.9077644, 93.86099): {'index_triangle': {98, 99, 7, 72, 9, 73, 76, 80, 17, 19, 116, 119, 86, 95, 88, 89, 94, 31}}

In [255]:
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 44
Number of edges: 120


In [256]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)